### 1. Fix đường dẫn

In [1]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent      # notebooks/ -> project/
SRC_DIR = PROJECT_ROOT / "src"

sys.path.append(str(SRC_DIR))

print("PROJECT_ROOT:", PROJECT_ROOT)
print("SRC_DIR added:", SRC_DIR)

PROJECT_ROOT: d:\NLP\NLP-project
SRC_DIR added: d:\NLP\NLP-project\src


### 2. Import thư viện và module

In [ ]:
import time     # Đo thời gian huấn luyện và đánh giá
import json     # Lưu kết quả (loss, BLEU, ...) ra file JSON

import torch
import torch.nn as nn
from torch.utils.data import DataLoader                     # Tạo batch dữ liệu
from torch.optim.lr_scheduler import ReduceLROnPlateau      # Giảm learning rate khi loss không cải thiện
from tqdm import tqdm                                       # Hiển thị progress bar
import sacrebleu                                            # Thư viện tính BLEU score

from config_loader import load_config
from data import (
    read_lines, tokenize_en, tokenize_fr,       # Đọc và tokenize dữ liệu
    build_vocab_from_token_lists,               # Xây dựng từ điển
    TranslationDataset, build_collate_fn,       # Dataset và hàm gộp batch
    PAD_TOKEN, SOS_TOKEN, EOS_TOKEN,            # Các token đặc biệt
)
from model import Encoder, Seq2SeqWithAttention, AttentionDecoder   # Các mô hình encoder-decoder

# Chọn thiết bị tính toán: GPU nếu có, ngược lại dùng CPU
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("DEVICE:", DEVICE)

DEVICE: cpu


### 3. Training & validation functions

In [ ]:
def train_epoch(
    model, dataloader, optimizer, criterion,
    teacher_forcing_ratio=0.5, clip=1.0, device="cpu"
):
    model.train()
    epoch_loss = 0.0

    for src, src_lens, tgt_in, tgt_out in tqdm(dataloader, desc="train"):
        # Đưa dữ liệu lên device (CPU/GPU)
        src, src_lens = src.to(device), src_lens.to(device)
        tgt_in, tgt_out = tgt_in.to(device), tgt_out.to(device)

        optimizer.zero_grad()   # Reset gradient

        # Forward pass với teacher forcing
        output = model(
            src, src_lens, tgt_in,
            teacher_forcing_ratio=teacher_forcing_ratio
        )
        vocab_size = output.size(-1)

        # Bỏ token <SOS> để căn chỉnh output và target
        output = output[:, 1:, :].contiguous()
        tgt_out = tgt_out[:, 1:].contiguous()

        # Tính loss trên toàn bộ chuỗi
        loss = criterion(
            output.view(-1, vocab_size),
            tgt_out.view(-1)
        )
        loss.backward()

        # Giới hạn gradient để tránh exploding gradient
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()    # Cập nhật trọng số

        epoch_loss += loss.item()

    return epoch_loss / len(dataloader)

def evaluate_epoch(model, dataloader, criterion, device="cpu"):
    model.eval()    # Chuyển sang chế độ đánh giá
    epoch_loss = 0.0

    with torch.no_grad():   # Không tính gradient khi validation
        for src, src_lens, tgt_in, tgt_out in tqdm(dataloader, desc="val"):
            src, src_lens = src.to(device), src_lens.to(device)
            tgt_in, tgt_out = tgt_in.to(device), tgt_out.to(device)

            # Không dùng teacher forcing khi đánh giá
            output = model(src, src_lens, tgt_in, teacher_forcing_ratio=0.0)
            vocab_size = output.size(-1)

            output = output[:, 1:, :].contiguous()
            tgt_out = tgt_out[:, 1:].contiguous()

            loss = criterion(
                output.view(-1, vocab_size),
                tgt_out.view(-1)
            )
            epoch_loss += loss.item()

    return epoch_loss / len(dataloader)

### 4. BLEU evaluation helpers

In [ ]:
def safe_tok_from_idx(vocab, idx: int):
    # Trả về token tương ứng với index, tránh lỗi out-of-range
    try:
        if 0 <= idx < len(vocab.itos):
            return vocab.itos[idx]
    except Exception:
        pass
    return "<unk>"  # Token không xác định

def generate_hyps_from_loader(model, dataloader, tgt_vocab, max_len=50, device="cpu"):
    model.eval()
    hyps, refs = [], []

    with torch.no_grad():   # Không tính gradient khi generate
        for src, src_lens, tgt_in, tgt_out in tqdm(dataloader, desc="generate"):
            src, src_lens = src.to(device), src_lens.to(device)

            # Greedy decoding: sinh từng token một cho đến <EOS> hoặc max_len
            preds = model.greedy_decode(
                src, src_lens,
                max_len=max_len,
                sos_idx=tgt_vocab.stoi[SOS_TOKEN],
                eos_idx=tgt_vocab.stoi[EOS_TOKEN],
            )
            
            # Chuyển output dự đoán từ index → câu
            for seq in preds:
                tokens = []
                for idx in seq:
                    if idx == tgt_vocab.stoi[EOS_TOKEN]:
                        break
                    tok = safe_tok_from_idx(tgt_vocab, int(idx))
                    if tok not in (PAD_TOKEN, SOS_TOKEN, EOS_TOKEN):
                        tokens.append(tok)
                hyps.append(" ".join(tokens))

            # Chuyển câu tham chiếu (ground truth) từ index → câu
            tgt_np = tgt_out.cpu().numpy()
            for line in tgt_np:
                tokens = []
                for idx in line:
                    if idx == tgt_vocab.stoi[EOS_TOKEN]:
                        break
                    tok = safe_tok_from_idx(tgt_vocab, int(idx))
                    if tok not in (PAD_TOKEN, SOS_TOKEN, EOS_TOKEN):
                        tokens.append(tok)
                refs.append(" ".join(tokens))

    return hyps, refs

def compute_corpus_bleu(hyps, refs):
    # Tính BLEU score ở mức corpus bằng sacreBLEU
    bleu = sacrebleu.corpus_bleu(hyps, [refs], force=True)
    return bleu.score

### 5. Load config & hyperparameters

In [ ]:
config = load_config()
device = DEVICE

train_cfg = config["training"]  # Nhóm tham số huấn luyện
BATCH_SIZE = train_cfg["batch_size"]
NUM_EPOCHS = train_cfg["num_epochs"]
LEARNING_RATE = train_cfg["learning_rate"]

# Teacher Forcing: cấu hình giá trị ban đầu + giảm dần theo epoch
BASE_TF = train_cfg["teacher_forcing"]
MIN_TF = train_cfg.get("min_teacher_forcing", 0.1)
TF_DECAY = train_cfg.get("tf_decay", 0.97)

CLIP = train_cfg["clip"]
MAX_LEN = train_cfg.get("max_len", 50)  # Độ dài tối đa khi inference/decoding

# Early stopping: dừng sớm nếu validation không cải thiện đủ trong 3 epoch liên tiếp
EARLY_PATIENCE = train_cfg.get("early_patience", 3)
EARLY_MIN_DELTA = train_cfg.get("early_min_delta", 1e-4)

model_cfg = config["model"] # Nhóm tham số kiến trúc mô hình
EMB_DIM = model_cfg["emb_dim"]
HID_DIM = model_cfg["hid_dim"]
N_LAYERS = model_cfg["n_layers"]
DROPOUT = model_cfg["dropout"]

data_cfg = config["data"]
data_dir = PROJECT_ROOT / data_cfg["data_dir"]

paths_cfg = config["paths"]
ckpt_path = PROJECT_ROOT / paths_cfg["checkpoint"]  # Đường dẫn lưu checkpoint model
ckpt_path.parent.mkdir(parents=True, exist_ok=True)

results_dir = PROJECT_ROOT / paths_cfg.get("results", "results")    # Thư mục lưu kết quả/biểu đồ
results_dir.mkdir(parents=True, exist_ok=True)

### 6. Load data & build vocab

In [ ]:
print("Loading data...")

# Đọc dữ liệu song ngữ từ các file train/val
train_src = read_lines(data_dir / "train.en")
train_tgt = read_lines(data_dir / "train.fr")
val_src = read_lines(data_dir / "val.en")
val_tgt = read_lines(data_dir / "val.fr")

# Tokenize câu nguồn (EN) và câu đích (FR)
train_src_tok = [tokenize_en(s) for s in train_src]
train_tgt_tok = [tokenize_fr(s) for s in train_tgt]
val_src_tok = [tokenize_en(s) for s in val_src]
val_tgt_tok = [tokenize_fr(s) for s in val_tgt]

MIN_FREQ = config["vocab"]["min_freq"]  # Ngưỡng tần suất tối thiểu để đưa token vào vocab

# Xây dựng từ điển chỉ dựa trên tập train để tránh data leakage
src_vocab = build_vocab_from_token_lists(train_src_tok, min_freq=MIN_FREQ)
tgt_vocab = build_vocab_from_token_lists(train_tgt_tok, min_freq=MIN_FREQ)

print("SRC vocab:", len(src_vocab))
print("TGT vocab:", len(tgt_vocab))

Loading data...
SRC vocab: 5893
TGT vocab: 6470


### 7. Dataset & DataLoader

In [ ]:
# Tạo dataset cho tập train và validation
train_ds = TranslationDataset(train_src_tok, train_tgt_tok, src_vocab, tgt_vocab)
val_ds = TranslationDataset(val_src_tok, val_tgt_tok, src_vocab, tgt_vocab)

# Collate function: padding chuỗi, tạo tensor và độ dài câu cho encoder
collate_fn = build_collate_fn(src_vocab, tgt_vocab)

# DataLoader cho huấn luyện (shuffle để tăng tính ngẫu nhiên)
train_loader = DataLoader(
    train_ds, batch_size=BATCH_SIZE,
    shuffle=True, collate_fn=collate_fn
)

# DataLoader cho validation (không shuffle)
val_loader = DataLoader(
    val_ds, batch_size=BATCH_SIZE,
    shuffle=False, collate_fn=collate_fn
)

### 8. Init model, optimizer, scheduler

In [ ]:
# Loss function: bỏ qua padding khi tính Cross-Entropy
pad_idx = tgt_vocab.stoi[PAD_TOKEN]
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

# Khởi tạo Encoder (LSTM)
enc = Encoder(
    len(src_vocab),
    EMB_DIM,
    HID_DIM,
    N_LAYERS,
    DROPOUT,
    pad_idx=src_vocab.stoi.get(PAD_TOKEN, 0)
)

# Khởi tạo Decoder có Attention
dec = AttentionDecoder(
    len(tgt_vocab),
    EMB_DIM,
    HID_DIM,
    N_LAYERS,
    DROPOUT
)

# Mô hình Seq2Seq kết hợp Encoder + Decoder + Attention
model = Seq2SeqWithAttention(enc, dec, device).to(device)

# Optimizer Adam cho toàn bộ tham số mô hình
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Scheduler: giảm learning rate khi validation loss không cải thiện
scheduler = ReduceLROnPlateau(
    optimizer,
    mode="min",
    factor=0.5,
    patience=2,
)

### 9. Training loop (core)

In [ ]:
# Log lại loss và BLEU theo từng epoch
train_loss_log, val_loss_log, val_bleu_log = [], [], []

best_bleu = -1.0
best_val_loss_report = float("inf")

best_val_loss_es = float("inf")
bad_epochs = 0

print("Training model...")

for epoch in range(1, NUM_EPOCHS + 1):
    t0 = time.time()

    # Teacher forcing decay theo epoch
    tf_ratio = max(
        MIN_TF,
        BASE_TF * (TF_DECAY ** (epoch - 1))
    )

    lr = optimizer.param_groups[0]["lr"]
    print(f"\n=== Epoch {epoch}/{NUM_EPOCHS} | TF={tf_ratio:.3f} | LR={lr:.6f} ===")

    train_loss = train_epoch(
        model, train_loader, optimizer, criterion,
        teacher_forcing_ratio=tf_ratio,
        clip=CLIP,
        device=device,
    )

    # Đánh giá trên validation set
    val_loss = evaluate_epoch(model, val_loader, criterion, device=device)

    # Sinh câu dịch và tính BLEU trên tập validation
    hyps, refs = generate_hyps_from_loader(
        model, val_loader, tgt_vocab,
        max_len=MAX_LEN,
        device=device,
    )
    val_bleu = compute_corpus_bleu(hyps, refs)

    train_loss_log.append(train_loss)
    val_loss_log.append(val_loss)
    val_bleu_log.append(val_bleu)

    # Cập nhật learning rate dựa trên validation loss
    scheduler.step(val_loss)

    elapsed = time.time() - t0
    gap = abs(train_loss - val_loss)

    print(
        f"Train Loss: {train_loss:.3f} | "
        f"Val Loss: {val_loss:.3f} | "
        f"Gap: {gap:.3f} | "
        f"Val BLEU: {val_bleu:.2f} | "
        f"Time: {elapsed:.1f}s"
    )

    # Theo dõi loss tốt nhất để báo cáo
    if val_loss < best_val_loss_report:
        best_val_loss_report = val_loss

    # Lưu checkpoint theo BLEU tốt nhất
    if val_bleu > best_bleu:
        best_bleu = val_bleu
        torch.save(
            {
                "model_state_dict": model.state_dict(),
                "src_itos": src_vocab.itos,
                "tgt_itos": tgt_vocab.itos,
                "config": config,
                "epoch": epoch,
                "val_bleu": best_bleu,
            },
            ckpt_path,
        )
        print("Saved best checkpoint (by Val BLEU)")

    # Early stopping theo validation loss
    if val_loss < best_val_loss_es - EARLY_MIN_DELTA:
        best_val_loss_es = val_loss
        bad_epochs = 0
    else:
        bad_epochs += 1

    if bad_epochs >= EARLY_PATIENCE:
        print("Early stopping triggered")
        break

Training model...

=== Epoch 1/12 | TF=0.500 | LR=0.000700 ===


generate: 100%|██████████| 16/16 [00:11<00:00,  1.33it/s]


Train Loss: 4.983 | Val Loss: 4.387 | Gap: 0.596 | Val BLEU: 2.75 | Time: 1294.5s
Saved best checkpoint (by Val BLEU)

=== Epoch 2/12 | TF=0.485 | LR=0.000700 ===


generate: 100%|██████████| 16/16 [00:07<00:00,  2.20it/s]


Train Loss: 4.041 | Val Loss: 3.846 | Gap: 0.195 | Val BLEU: 8.39 | Time: 1202.7s
Saved best checkpoint (by Val BLEU)

=== Epoch 3/12 | TF=0.470 | LR=0.000700 ===


...

Train Loss: 2.329 | Val Loss: 2.905 | Gap: 0.576 | Val BLEU: 22.51 | Time: 1005.6s
Saved best checkpoint (by Val BLEU)

=== Epoch 11/12 | TF=0.369 | LR=0.000700 ===


generate: 100%|██████████| 16/16 [00:08<00:00,  1.88it/s]


Train Loss: 2.252 | Val Loss: 2.874 | Gap: 0.622 | Val BLEU: 24.13 | Time: 994.4s
Saved best checkpoint (by Val BLEU)

=== Epoch 12/12 | TF=0.358 | LR=0.000700 ===


generate: 100%|██████████| 16/16 [00:07<00:00,  2.11it/s]


Train Loss: 2.173 | Val Loss: 2.846 | Gap: 0.673 | Val BLEU: 24.47 | Time: 1259.7s
Saved best checkpoint (by Val BLEU)


### 10. Save metrics

In [ ]:
metrics = {
    "train_loss": train_loss_log,
    "val_loss": val_loss_log,
    "val_bleu": val_bleu_log,
}

# Lưu metrics ra file JSON để phục vụ vẽ biểu đồ và báo cáo
metrics_path = results_dir / "metrics.json"
with open(metrics_path, "w", encoding="utf-8") as f:
    json.dump(metrics, f, indent=2)

print("Saved metrics to:", metrics_path)

print(
    f"\nTraining finished.\n"
    f"Best Val Loss = {best_val_loss_report:.3f}\n"
    f"Best Val BLEU = {best_bleu:.2f}"
)

Saved metrics to: d:\NLP\NLP-project\results\metrics.json

Training finished.
Best Val Loss = 2.846
Best Val BLEU = 24.47
